In [1]:
import import_ipynb
from collections import OrderedDict
from random import randint

import os
import sys
sys.path.append('../../')  # Go up two folders to the project root

from structures.Lexicon import Lexicon
from structures.LexiconRow import LexiconRow

importing Jupyter notebook from D:\GitHub\developing\tests\Structures\../..\structures\Lexicon.ipynb
importing Jupyter notebook from D:\GitHub\developing\tests\Structures\../..\structures\LexiconRow.ipynb
importing Jupyter notebook from D:\GitHub\developing\tests\Structures\../..\structures\DocumentIndex.ipynb
importing Jupyter notebook from D:\GitHub\developing\tests\Structures\../..\utilities\General_Utilities.ipynb
importing Jupyter notebook from D:\GitHub\developing\tests\Structures\../..\structures\DocumentIndexRow.ipynb
importing Jupyter notebook from D:\GitHub\developing\tests\Structures\../..\building_data_structures\CollectionStatistics.ipynb


In [2]:
import pytest
import ipytest

ipytest.autoconfig()

In [3]:
%%ipytest

def test_lexicon_add_term():
    capacity = 5
    lexicon = Lexicon(capacity)
    assert lexicon.is_empty()
    
    lex_row = LexiconRow("test_term", 10)
    lexicon.add_term(lex_row)

    key = "test_term".ljust(lex_row.MAX_TERM_LENGTH)

    assert len(lexicon._vocabulary) == 1
    assert lexicon._vocabulary == lexicon.get_structure()
    
    assert lexicon.get_terms(key).term == key
    assert lexicon.get_terms(key).term == lex_row.term
    assert lexicon.get_terms(key).dft == lex_row.dft

    assert not lexicon.is_empty()

    # Add 100 row to lexicon and test that:
    # - size remains lower or equal than 5 all the time
    # - structure follow the "LRU" cache replacement policy
    lexicon.add_term(LexiconRow("term_1", 1))
    lexicon.add_term(LexiconRow("term_2", 2))
    lexicon.add_term(LexiconRow("term_3", 3))
    lexicon.add_term(LexiconRow("term_4", 4))
    for i in range(5, 100):
        random_term = f"term_{i}"
        lex_row = LexiconRow(random_term, i)
    
        lexicon.add_term(lex_row)
        assert len(lexicon.get_structure()) <= 5
        
        counter = capacity - 1
        for elem in lexicon.get_structure().values():
            assert elem.term == f"term_{int(i - counter)}".ljust(lex_row.MAX_TERM_LENGTH)
            counter = counter - 1   


    lexicon.clear_structure()
    assert lexicon.is_empty()
    
    found_entry = lexicon.find_entry("random_word_not_present_in_lexicon")
    assert found_entry == None
    found_entry = lexicon.find_entry("happiness")
    assert found_entry.term == "happiness".ljust(lex_row.MAX_TERM_LENGTH)

    assert lexicon.is_empty()
    lex_row = lexicon.get_entry("happiness")
    assert not lexicon.is_empty()

    key = "happiness".ljust(lex_row.MAX_TERM_LENGTH)
    assert lexicon.get_terms(key).term == key
    assert lexicon.get_terms(key).term == lex_row.term
    assert lexicon.get_terms(key).dft == lex_row.dft

.                                                                                            [100%]
1 passed in 0.03s
